### How Do We Interface With External Resources?

In [15]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install openapi-python-client
!{sys.executable} -m pip install openapi-python-generator


  Using cached pydantic-2.8.0-py3-none-any.whl.metadata (123 kB)
Using cached pydantic-2.8.0-py3-none-any.whl (423 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.17
    Uninstalling pydantic-1.10.17:
      Successfully uninstalled pydantic-1.10.17
  Using cached openapi_python_generator-0.5.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached httpx-0.23.3-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-1.10.17-cp312-cp312-macosx_11_0_arm64.whl.metadata (151 kB)
  Using cached httpcore-0.16.3-py3-none-any.whl.metadata (16 kB)
Using cached openapi_python_generator-0.5.0-py3-none-any.whl (22 kB)
Using cached httpx-0.23.3-py3-none-any.whl (71 kB)
Using cached pydantic-1.10.17-cp312-cp312-macosx_11_0_arm64.whl (2.2 MB)
Using cached httpcore-0.16.3-py3-none-any.whl (69 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.0
    Uninstalling pydantic-2.8.0:
      Successfully uninstalled pydantic-2.8.0
  Attempting un

In [1]:
import requests

url = "https://v2.jokeapi.dev/joke/Any?safe-mode&type=twopart"

response = requests.get(url)

print(response.json()["setup"])
print(response.json()["delivery"])

Which is faster, Hot or cold?
Hot, because you can catch a cold


In [2]:
response.json()

{'error': False,
 'category': 'Pun',
 'type': 'twopart',
 'setup': 'Which is faster, Hot or cold?',
 'delivery': 'Hot, because you can catch a cold',
 'flags': {'nsfw': False,
  'religious': False,
  'political': False,
  'racist': False,
  'sexist': False,
  'explicit': False},
 'safe': True,
 'id': 279,
 'lang': 'en'}

In [3]:
import requests
def give_joke(category : str):
    """
    Joke categories. Supports: Any, Misc, Programming, Pun, Spooky, Christmas.
    """

    url = f"https://v2.jokeapi.dev/joke/{category}?safe-mode&type=twopart"
    response = requests.get(url)
    print(response.json()["setup"])
    print(response.json()["delivery"])

In [4]:
USER_QUERY = "Hey! Can you get me a joke for this december?"

In [5]:
from utils import query_raven

raven_functions = \
f'''
def give_joke(category : str):
    """
    Joke categories. Supports: Any, Misc, Programming, Dark, Pun, Spooky, Christmas.
    """

User Query: {USER_QUERY}<human_end>
'''
call = query_raven(raven_functions)

In [6]:
exec(call)

What says Oh Oh Oh?
Santa walking backwards!


#### Writing A Tool That Uses OpenAPI APIs

In [7]:
#!wget https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml

In [8]:
import yaml
import json

# Read the content of the file
with open('openapi.yml', 'r') as file:
    file_content = file.read()
file_content = file_content.replace("int\n", "number\n")
file_content = file_content.replace("float\n", "number\n")
data = yaml.safe_load(file_content)

data["servers"] = [{"url":"https://api.open-meteo.com"}]

with open('openapi.json', 'w') as file:
    json_content = json.dump(data, file)

In [16]:
!openapi-python-generator openapi.json ./api_specification_main/


Generating data from openapi.json


In [17]:
from api_specification_main.services.WeatherForecastAPIs_service\
    import get_v1forecast

In [18]:
user_query = "Hey how is the current weather and windspeed in New York?"

In [19]:
import inspect
signature = inspect.signature(get_v1forecast)
docstring = \
'''
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on preference.
'''

raven_prompt = \
f'''
Function:
{get_v1forecast.__name__}{signature}
"""{docstring}"""

User Query: {user_query}<human_end>'''

print (raven_prompt)


Function:
get_v1forecast(latitude: float, longitude: float, hourly: Optional[List[str]] = None, daily: Optional[List[str]] = None, current_weather: Optional[bool] = None, temperature_unit: Optional[str] = None, wind_speed_unit: Optional[str] = None, timeformat: Optional[str] = None, timezone: Optional[str] = None, past_days: Optional[int] = None, api_config_override: Optional[api_specification_main.api_config.APIConfig] = None) -> Dict[str, Any]
"""
Requires the latitude and longitude.
Set current_weather to True to get the weather.
Set hourly or daily based on preference.
"""

User Query: Hey how is the current weather and windspeed in New York?<human_end>


In [20]:
from utils import query_raven
call = query_raven(raven_prompt)
print (call)

get_v1forecast(latitude=40.7128, longitude=-74.0060, current_weather=True)


In [21]:
eval(call)

{'latitude': 40.710335,
 'longitude': -73.99309,
 'generationtime_ms': 0.07009506225585938,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 32.0,
 'current_weather_units': {'time': 'iso8601',
  'interval': 'seconds',
  'temperature': '°C',
  'windspeed': 'km/h',
  'winddirection': '°',
  'is_day': '',
  'weathercode': 'wmo code'},
 'current_weather': {'time': '2024-07-03T22:30',
  'interval': 900,
  'temperature': 25.8,
  'windspeed': 19.0,
  'winddirection': 147,
  'is_day': 1,
  'weathercode': 3}}